In [1]:
from simpletransformers.classification import ClassificationModel
import pandas as pd


# Train and Evaluation data needs to be in a Pandas Dataframe of two columns. The first column is the text with type str, and the second column is the label with type int.
train_data = [['属于类一的例句', 1], ['属于类二的例句', 0]]
train_df = pd.DataFrame(train_data)

eval_data = [['属于类一的例句2', 1], ['属于类二的例句1', 0]]
eval_df = pd.DataFrame(eval_data)

# Create a ClassificationModel
model = ClassificationModel('bert', 'bert-base-chinese', use_cuda=False, args={'overwrite_output_dir': True}) # You can set class weights by using the optional weight argument

# Train the model
model.train_model(train_df)

# Evaluate the model
result, model_outputs, wrong_predictions = model.eval_model(eval_df)

OSError: Unable to load weights from pytorch checkpoint file. If you tried to load a PyTorch model from a TF 2.0 checkpoint, please set from_tf=True. 

In [2]:
import sys
sys.executable

'/opt/anaconda3/bin/python'

In [2]:
import sqlite3